## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3
import zipfile

#### Genre analysis-what genres are consistent in box office

In [2]:
#import the movie info tsv file
df = pd.read_csv("rt.movie_info.tsv", delimiter="\t")
df.head(6)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures


In [3]:
#most watched genres group by genre
df["genre"].value_counts()

genre
Drama                                                                                     151
Comedy                                                                                    110
Comedy|Drama                                                                               80
Drama|Mystery and Suspense                                                                 67
Art House and International|Drama                                                          62
                                                                                         ... 
Art House and International|Drama|Sports and Fitness                                        1
Comedy|Documentary|Musical and Performing Arts|Special Interest                             1
Comedy|Cult Movies|Mystery and Suspense|Science Fiction and Fantasy                         1
Action and Adventure|Art House and International|Mystery and Suspense|Special Interest      1
Comedy|Drama|Kids and Family|Sports and Fitness       

###### This shows Drama is most frequently watched movie according to the dataset genre followed by comedy then a combination of both comedy and drama

In [4]:
#output containing basic summary statistics about the data 
df.describe()

,id
count,1560.000000
mean,1007.303846
std,579.164527
min,1.000000
25%,504.750000
50%,1007.500000
75%,1503.250000
max,2000.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [6]:
#converting runtime to int
df['runtime'] = df['runtime'].str.extract('(\d+)').fillna(0).astype(int)

In [7]:
# average runtime of all mvoies genres
mean_ = df["runtime"].mean()
median_ = df["runtime"].median()
print(f"mean:{mean_}")
print(f"mean:{median_}")

mean:101.96858974358975
mean:100.0


The mean and median shows a good number of movies lie around 100minutes meaning more emphasis on making movies that last close to about 100 minutes

In [8]:
#finding 
drama = df[(df['genre'] == 'Drama') & (df['runtime'] > 0)]
drama.head()
len(drama)

146

This dataset shows movie that contain drama,or drama and a combination of a different movie


### Movie budget- What is the cost of producing a movie

##### This part looks at what the company has to spend and what would be gained in domestic stage, international or worldwide and is it making a profit or a loss

In [9]:
budget_df= pd.read_csv("tn.movie_budgets.csv",index_col=0)
budget_df.head(10)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"


In [10]:
#removing $ sign and (,) comma for statistical calculation
budget_df['production_budget'] = budget_df['production_budget'].str.replace('[\$,]', '', regex=True)

budget_df['domestic_gross'] = budget_df['domestic_gross'].str.replace('[\$,]', '', regex=True)

budget_df['worldwide_gross'] = budget_df['worldwide_gross'].str.replace('[\$,]', '', regex=True)
budget_df.head(2)

,release_date,movie,production_budget,domestic_gross,worldwide_gross
id,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875


In [11]:
#converting to int64
budget_df['production_budget'] = budget_df['production_budget'].astype('int64')
budget_df['domestic_gross'] = budget_df['domestic_gross'].astype('int64')
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].astype('int64')
budget_df.head(5)
type(budget_df['worldwide_gross'])

pandas.core.series.Series

In [12]:
#calculating international gross
budget_df['international_gross'] = (budget_df['worldwide_gross']) - (budget_df['domestic_gross'])
budget_df.head(10)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross
id,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365
6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1116648995
7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,1369318718
8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,654000000
9,"Nov 17, 2017",Justice League,300000000,229024295,655945209,426920914


In [13]:
budget_df["gross_difference"] = (budget_df["international_gross"]) - (budget_df["domestic_gross"])
budget_df.drop(3)
budget_df.head(5)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,gross_difference
id,,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,1255330029
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,563536125
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,64237650
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,485002227
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,76358983


In [16]:
budget analysis
"""from the above claculation we can see that more of the movie selling 
is from international audience but 

"""

SyntaxError: invalid syntax (2082408318.py, line 1)

In [18]:
#check for duplicates
budget_df["movie"].duplicated(keep = False).sort_values()
budget_df.head(5)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,gross_difference,profits
id,,,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,1255330029,2351345279
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,563536125,635063875
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,64237650,-200237650
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,485002227,1072413963
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,76358983,999721747


In [17]:
#calculating profit
budget_df["profits"] = (budget_df['worldwide_gross'])-(budget_df['production_budget'])
budget_df.sort_values(by=["production_budget"])
budget_df.head(10)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,international_gross,gross_difference,profits
id,,,,,,,,
1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2015837654,1255330029,2351345279
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000,563536125,635063875
3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,107000000,64237650,-200237650
4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,944008095,485002227,1072413963
5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365,76358983,999721747
6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1116648995,179986770,1747311220
7,"Apr 27, 2018",Avengers: Infinity War,300000000,678815482,2048134200,1369318718,690503236,1748134200
8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,300000000,309420425,963420425,654000000,344579575,663420425
9,"Nov 17, 2017",Justice League,300000000,229024295,655945209,426920914,197896619,355945209


In [22]:
budget_df= pd.read_csv("tmdb.movies.csv",index_col=0)
budget_df.head(5)

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


#### dataset to analyze successful movies in terms of actors, directors, and other key crew members.

In [ ]:
df = pd.read_csv("rt.movie_info.tsv", delimiter="\t")
df.head(6)

In [32]:
reviews_df = pd.read_csv("rt.reviews.tsv", delimiter="\t", encoding='latin1')
reviews_df.head(6)

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
5,3,... Cronenberg's Cosmopolis expresses somethin...,NaN,fresh,Michelle Orange,0,Capital New York,"September 11, 2017"
